In [5]:
import open3d as o3d
import numpy as np
from sklearn.neighbors import NearestNeighbors
import os
import random

In [2]:
datasetPath = "/PublicData/CadDataset/a1.0.0_10"
part = "140294_f659b875"
name = "assembly.obj"
file_path = os.path.join(datasetPath,part,name)

In [3]:
num_sample_points = 300000

mesh = o3d.io.read_triangle_mesh(file_path)
mesh.compute_vertex_normals()

TriangleMesh with 111342 points and 122916 triangles.

In [4]:
pcd = mesh.sample_points_poisson_disk(number_of_points=num_sample_points)
print(f"Sampled {len(pcd.points)} points")

Sampled 300000 points


In [6]:
# 降采样，减少点数，提高聚类速度
voxel_size = 0.01  # 可调节
pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
print(f"Downsampled to {len(pcd_down.points)} points")

Downsampled to 300000 points


In [7]:
# DBSCAN聚类
labels = np.array(
    pcd_down.cluster_dbscan(eps=0.02, min_points=20, print_progress=True)
)
num_clusters = labels.max() + 1
print(f"DBSCAN found {num_clusters} clusters")

Precompute neighbors.[========================================] 100%
DBSCAN found 0 clusters


In [8]:
# 上色
colors = np.zeros((len(labels), 3))
for label in np.unique(labels):
    if label == -1:
        colors[labels == label] = [0, 0, 0]
    else:
        colors[labels == label] = [random.random(), random.random(), random.random()]
pcd_down.colors = o3d.utility.Vector3dVector(colors)

In [9]:
# 保存和显示
o3d.io.write_point_cloud("fast_dbscan_segmentation.ply", pcd_down)

True